In [ ]:
!pip install hazm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from __future__ import unicode_literals
from hazm import *
from tqdm import tqdm
import json
with open("/content/drive/MyDrive/IR_data_news_12k.json", "r") as read_file:
    data = json.load(read_file)

In [ ]:
def print_result(docs):
  for doc in docs:
    print('title:',data[doc]['title'])
    print('url:', data[doc]['url'])
    print('--------------------------------------------------------')

In [ ]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()

stop_words = stopwords_list()

def preprocess(text):
    text = normalizer.normalize(text)
    # print(text)
    words = word_tokenize(text)
    # print(words)
    
    words[:] = [x for x in words if not (x in stop_words)]

    for i in range(len(words)):
        words[i] = lemmatizer.lemmatize(words[i])

    return words


for doc, value in tqdm(data.items()):
    data[doc]['title'] = preprocess(data[doc]['title'])
    data[doc]['content'] = preprocess(data[doc]['content'])

100%|██████████| 12202/12202 [00:30<00:00, 396.20it/s]


In [ ]:
print(data["0"])

{'title': ['اعلام', 'زمان', 'قرعه', 'کشید#کش', 'جام', 'باشگاه', 'فوتسال', 'آسیا'], 'content': ['گزارش', 'خبرگزاری', 'فارس', '،', 'کنفدراسیون', 'فوتبال', 'آسیا', '(', 'AFC', ')', 'نامه', 'رسم', 'فدراسیون', 'فوتبال', 'ایران', 'باشگاه', 'گیتی', 'پسند', 'زمان', 'قرعه', 'کشید#کش', 'جام', 'باشگاه', 'فوتسال', 'آسیا', 'رسما', 'اعلام', '.', 'اساس', '۲۵', 'فروردین', 'ماه', '۱۴۰۱', 'مراسم', 'قرعه', 'کشید#کش', 'جام', 'باشگاه', 'فوتسال', 'آسیا', 'مالزی', 'برگزار', '.', 'باشگاه', 'گیتی', 'پسند', 'بعنوان', 'قهرمان', 'فوتسال', 'ایران', 'سال', '۱۴۰۰', 'مسابقات', '.', 'گیتی', 'پسند', 'تجربه', '۳', 'دوره', 'حضور', 'جام', 'باشگاه', 'فوتسال', 'آسیا', 'دوره', 'فینال', 'مسابقات', 'عنوان', 'قهرمان', 'مقام', 'دوم', 'بدست', 'آورد#آور', '.', 'انتهای', 'پیام/'], 'tags': ['اعلام زمان', 'قرعه\u200cکشی', 'قرعه\u200cکشی جام', 'قرعه\u200cکشی جام باشگاه\u200cهای فوتسال', 'ای اف سی', 'گیتی پسند'], 'date': '3/15/2022 5:59:27 PM', 'url': 'https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسا

In [ ]:
title_pos_index = {}
content_pos_index = {}

for doc in data:       
    for pos, term in enumerate(data[doc]["content"]):
      if term in content_pos_index:
        content_pos_index[term][0] = content_pos_index[term][0] + 1

        if doc in content_pos_index[term][1]:
          content_pos_index[term][1][doc].append(pos)
                
        else:
          content_pos_index[term][1][doc] = [pos]
      else:
        content_pos_index[term] = []
        content_pos_index[term].append(1)
        content_pos_index[term].append({})     
        content_pos_index[term][1][doc] = [pos]

In [ ]:
# print(title_pos_index["سلام"])
print(content_pos_index["سلام"])

[364, {'314': [82], '860': [183], '964': [300], '1102': [90], '1221': [80], '1312': [82], '1322': [708], '1382': [61, 70, 74, 79], '1585': [53], '1858': [19], '1903': [115], '1961': [20], '1963': [22], '2046': [45], '2276': [260], '2313': [36], '2563': [281], '2617': [25], '3422': [146], '3509': [34, 40], '3537': [157], '3662': [294], '4005': [60, 63], '4033': [100], '4120': [38], '4152': [157], '4169': [105], '4244': [65], '4455': [98], '4592': [34], '4607': [61], '4873': [272], '5068': [42], '5116': [18], '5544': [168], '5800': [76], '6675': [94], '6847': [727], '6895': [72], '6915': [1956, 2193], '6967': [13, 44], '6988': [38], '7016': [164], '7018': [136], '7087': [32], '7143': [42], '7181': [44], '7219': [55], '7231': [38], '7234': [48], '7241': [24], '7253': [107], '7259': [505, 510], '7260': [233, 236], '7272': [286], '7331': [132], '7334': [53], '7423': [125], '7517': [60], '7541': [32], '7554': [186], '7575': [55], '7579': [112], '7694': [48], '7739': [1694, 1769], '7758': [21

In [ ]:
import itertools

minimum_candidates = 5


def boolean_include_query(term):
  if term not in content_pos_index:
    return []
  candidates = content_pos_index[term][1]
  return [c[0] for c in sorted(candidates.items(), key=lambda x: len(x[1]), reverse=True)]

def boolean_exclude_query(term, candidates):
  exclude = boolean_include_query(term)
  return [x for x in candidates if not (x in exclude)]

def candidates_intersection(candidates):
  res = candidates[0]
  for i in range(1, len(candidates)):
    res = [value for value in res if value in candidates[i]]
  return res
    
def boolean_query(query):
  query_words = preprocess(query)

  to_exclude = []
  candidates = []

  for index in range(len(query_words)):
    if query_words[index] == "!":
      index = index + 1
      to_exclude.append(query_words[index])
    elif index == 0 or query_words[index-1] != "!":
      candidates.append(boolean_include_query(query_words[index]))

  candidates = candidates_intersection(candidates)
  for word in to_exclude:
    candidates = boolean_exclude_query(word, candidates)

  if len(candidates) >= minimum_candidates:
    return candidates

  for min_words in range(len(query_words)-1, len(query_words)-3, -1): 
    query.replace(" ! ", " !")
    new_query_words = list(itertools.combinations(query.split(), min_words))

    for subset in new_query_words:
      query_words = preprocess(' '.join(subset))
      print(query_words)
      for index in range(len(query_words)):
        if query_words[index] == "!":
          index = index + 1
          to_exclude.append(query_words[index])
        elif index == 0 or query_words[index-1] != "!":
          candidates.append(boolean_include_query(query_words[index]))

      candidates = candidates_intersection(candidates)
      for word in to_exclude:
        candidates = boolean_exclude_query(word, candidates)

      if len(candidates) > minimum_candidates:
        return candidates
  
  return []
  

In [ ]:
def positional_intersect(p1, p2, candidates):
  pip1 = content_pos_index[p1][1]
  pip2 = content_pos_index[p2][1]
  res = []
  for doc1, indexes1 in pip1.items():
    for doc2, indexes2 in pip2.items():
      if int(doc2) > int(doc1):
        break
      elif doc1 == doc2 and doc1 in candidates:
        for term1_index in indexes1:
          for term2_index in indexes2:
            if term1_index == term2_index-1:
              if not(doc1 in res):
                res.append(doc1)
            elif term1_index > term2_index:
              continue
            else:
              break
  return res

def phrase_query(query):
  query_words = preprocess(query)

  candidates = [c for c in content_pos_index[query_words[0]][1]]

  for i in range(1, len(query_words)):
    candidates = positional_intersect(query_words[i-1], query_words[i], candidates)

  return candidates


In [ ]:
query = "تحریم‌های آمریکا علیه ایران"

print('boolean query:')
for doc in boolean_query(query)[:5]:
  print(data[doc]['url'])
print()
print('phrase query:')
for doc in phrase_query(query):
  print(data[doc]['url'])

boolean query:
https://www.farsnews.ir/news/14001222000450/توضیحات-یک-منبع-آگاه-درباره-وقفه-مذاکرات-وین
https://www.farsnews.ir/news/14000908000357/هتل-کوبورگ-زیر-ذره‌بین-بهارستان-تاکید-وکلای-ملت-بر-انتفاع-اقتصادی
https://www.farsnews.ir/news/14000819000731/مذاکره‌کنندگان-مراقب-زنجیره-تحریم‌ها-باشند-حداقل-4-تا-6-ماه-برای
https://www.farsnews.ir/news/14000821000097/راهبرد-مذاکره-هسته‌ای-نداریم-چگونه-تأمین-کننده-منافع-کشور-خواهد-بود-
https://www.farsnews.ir/news/14000803000676/اهرم‌سازی-از-افغانستان-در-برجام-نقطه-عزیمت-آمریکا-در-مذاکرات-جامع-با

phrase query:
https://www.farsnews.ir/news/14001222000450/توضیحات-یک-منبع-آگاه-درباره-وقفه-مذاکرات-وین
https://www.farsnews.ir/news/14001024000193/نباید-مانند-دولت-گذشته-در-مذاکرات-افراط-کرد-توافق-موقت-راهبرد-اصلی


In [ ]:
import re

def process_query(query):
  phrase_queries = re.findall('"([^"]*)"', query)
  bool_query = re.sub('"([^"]*)"', '', query)


  candidates = []
  if len(bool_query) != 0:
    candidates = [boolean_query(bool_query)]

  if len(phrase_queries) == 0:
    return candidates[0]

  for phrase in phrase_queries:
    candidates.append(phrase_query(phrase))

  return candidates_intersection(candidates)

In [ ]:
print_result(process_query('"تحریم هسته‌ای" آمریکا ! ایران'))
print_result(process_query('آمریکا ! ایران'))
print_result(process_query('"تحریم هسته‌ای"'))

title: ['آمریکا', 'بحران\u200cآفرین', 'حیات', 'ادامه']
url: https://www.farsnews.ir/news/14001213000089/آمریکا-با-بحران‌آفرینی-به-حیات-خود-ادامه-می-دهد
--------------------------------------------------------
title: ['سود', 'مافیا', 'اسلحه\u200cسازی', 'آمریکا', 'ناامن', 'جهان']
url: https://www.farsnews.ir/news/14001211000898/سود-مافیای-اسلحه‌سازی-آمریکا-در-ناامن-بودن-جهان-است
--------------------------------------------------------
title: ['آمریکا', 'دولت', 'متحد', 'زمان', 'اضطرار', 'گذاشت#گذار']
url: https://www.farsnews.ir/news/14001211000321/آمریکا-دولت‌های-متحد-خود-را-در-زمان-اضطرار-تنها-می‌گذارد
--------------------------------------------------------
title: ['خباثت', 'آمریکا', 'برجا', 'روسیه', 'چین', 'کشور', 'بدبین']
url: https://www.farsnews.ir/news/14001217000665/خباثت‌های-آمریکا-در-برجام-روسیه-و-چین-را-هم-به-این-کشور-بدبین-کرد
--------------------------------------------------------
title: ['ماهیت', 'استکبار', 'آمریکا', 'دنیا', 'معرف', 'کنیم/', 'تقدیر', 'اقدام', 'سپاه', 'جلوگ

In [ ]:
import math

def tf(term, doc):
  if doc in content_pos_index[term][1]:
    term_freq = len(content_pos_index[term][1][doc])
    return 1 + math.log(term_freq, 10)
  
  return 1

def tf_phrase_query(term, query):
  query_words = preprocess(query)
  # if no word in the query matches the term, then we return 1 + log(1) = 1
  term_freq = 1
  for word in query_words:
    if word == term:
      term_freq += 1

  return 1 + math.log(term_freq, 10)


def idf(term):
  if term not in content_pos_index:
    return 0

  df = len(content_pos_index[term][1])
  N = len(data)

  return math.log(N/df, 10)


def tf_idf(term, doc):
  return tf(term, doc)*idf(term)

In [ ]:
query_words_example = 'سلام سلام خوبی'
print(tf_phrase_query('سلام', query_words_example))
print(tf_phrase_query('خوبی', query_words_example))
print(idf("فوتبال"))

1.4771212547196624
1.0
0.5656855491368863


In [ ]:
def create_doc_vectors():
  doc_vectors = {term: [0]*len(data) for term in content_pos_index }
  for term in content_pos_index:
    for doc in content_pos_index[term][1]:    
      doc_vectors[term][int(doc)] = tf_idf(term, doc)

  return doc_vectors

doc_vectors = create_doc_vectors()

In [ ]:
print(type(data))
print(content_pos_index['آسیا'])

print(doc_vectors['آسیا'])

# print(doc_vectors["سلام"])
# print(len(doc_vectors["سلام"]))

# print(doc_vectors["ایران"])
# print(len(doc_vectors["ایران"]))

<class 'dict'>
[2348, {'0': [6, 24, 39, 63], '8': [155, 162, 189, 194, 302, 352], '13': [166, 255, 269], '29': [626], '30': [16, 23, 58], '34': [134], '50': [550, 557], '54': [208, 223], '55': [348], '56': [66], '60': [59], '65': [22], '68': [14, 31, 119, 122, 185, 200, 208, 249, 256, 269, 288, 348, 372], '70': [11, 96], '71': [155], '84': [33], '86': [481], '91': [17], '93': [41, 51, 65], '104': [153], '122': [14, 44, 50], '130': [11, 70], '140': [239], '141': [69], '142': [55, 61, 112, 123, 167, 198, 224, 336], '143': [17, 88, 101, 116, 168], '148': [194, 263, 544, 597], '155': [192], '163': [2329, 2399, 2416, 2434, 2454], '181': [501], '182': [83], '184': [141], '191': [203], '202': [72, 81], '210': [13], '221': [12], '232': [33, 52, 100], '281': [114], '286': [71], '301': [8, 45], '304': [345], '305': [26], '320': [170], '322': [21, 254], '337': [17, 78, 93], '345': [14], '382': [17, 55], '387': [43, 215], '391': [69], '395': [50, 62, 108, 115], '406': [18, 26], '408': [95], '409':

In [ ]:
def query_to_vector(query):
  # query_vector = {"term1": tfidf(term1, q), "term2": tfidf(term2, q), ...}
  return [tf_phrase_query(term, query) * 
          idf(term) for term in content_pos_index]


def doc_to_vector(doc):
  doc_vector = [0 for _ in range(len(doc_vectors))]
  index = 0
  for term in doc_vectors:
    doc_vector[index] = doc_vectors[term][int(doc)]
    index += 1

  return doc_vector

In [ ]:
print(doc_to_vector("0"))
print(query_to_vector('فوتبال ایران آمریکا'))
print(query_to_vector('تحریم هسته‌ای آمریکا'))

[0.06590400838180586, 0.0072135849098017395, 0.0026060246030320355, 0.0035021056412384215, 1.7725638002872153, 0.73597386754074, 1.699330083821194, 0.47027070780878566, 1.7986292907261423, 0.46964053432665287, 1.0631785610226572, 1.1817154753776875, 0.9068792294911809, 0.5844767949584315, 1.1682513447626577, 3.5527569505186203, 3.310720584610637, 0.6836539510460212, 2.725924480461501, 1.4544839753999852, 1.2777346755838257, 2.8708351031283135, 1.9161693052614113, 0.6475723612358067, 0.0021129687610350494, 0.7552062396356363, 1.3468586762062766, 1.805397653408641, 0.7566264984922992, 1.2090596747865945, 1.2877803752110997, 2.4429783441701813, 0.6312814994765407, 1.9497104534999616, 1.055890835434245, 0.46494315607573844, 1.1817154753776875, 1.1461501599337125, 1.207909225155162, 0.8453828699847242, 1.4406801094856345, 0.45024412545764436, 1.5732134205884296, 0.49214899184456257, 1.11330316705667, 2.057047242971159, 1.7381261576082077, 0.9937103759722694, 0.004723750559019523, 0.02070196

In [ ]:
def dot_product(a, b):
  result = 0
  for i in range(0, len(a)):
    result += (a[i] * b[i])

  return result


def vector_len(a):
  sum = 0
  for element in a:
    sum += element * element

  return math.sqrt(sum)


def cosine_similarity(query_vector, doc_vector):
  return dot_product(query_vector, doc_vector) / (vector_len(query_vector) * 
                                                  vector_len(doc_vector))

In [ ]:
print("Comparing the below document with 2 different queries using cosine similarity:")
print()

print(data["0"]["url"])
doc_vector_example = doc_to_vector("0")

query_vector_example1 = query_to_vector('فوتبال ایران آمریکا')
query_vector_example2 = query_to_vector('دانشگاه آمریکا')

print(cosine_similarity(query_vector_example1, doc_vector_example))
print(cosine_similarity(query_vector_example2, doc_vector_example))

Comparing the below document with 2 different queries using cosine similarity:

https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسال-آسیا
0.010236994163908192
0.010212922750719285


In [ ]:
def cosine_score(query):
  query_vector = query_to_vector(query)
  # eliminate documents that don't contain the query's words
  candidate_docs = boolean_query(query) 

  res = []
  for doc in candidate_docs:
      res.append((doc, 
                  cosine_similarity(query_vector, doc_to_vector(doc))))
  res.sort(key=lambda a: a[1])
  return [element[0] for element in res[:k]]

In [ ]:
k = 5

print_result(cosine_score('فوتبال ایران آمریکا'))
print_result(cosine_score('ایزان فوتبال آمریکا'))

title: ['واکنش', 'رسانه', 'صهیونیست', 'خشم', 'ایرانی\u200cها', 'اقدام', 'ناشایست', 'مهدوی', 'کیا+عکس']
url: https://www.farsnews.ir/news/14001001000459/واکنش-رسانه-صهیونیست-به-خشم-ایرانی‌ها-از-اقدام-ناشایست-مهدوی-کیا-عکس
--------------------------------------------------------
title: ['حمایت', 'فدراسیون', 'فوتبال', 'اقدام', 'مهدوی\u200cکیا/', 'کامرانی\u200cفر', ':', 'حواشی', 'غیرورزشی', 'مسابقه', 'مدیریت']
url: https://www.farsnews.ir/news/14001008000272/حمایت-فدراسیون-فوتبال-از-اقدام-مهدوی‌کیا-کامرانی‌فر-او-حواشی
--------------------------------------------------------
title: ['اصول', ':', 'فدراسیون', 'فوتبال', 'جمهوری', 'اسلام', 'ایران', 'جزیره', 'مستقل/', 'گفتار', 'ساختارشکنانه', 'فدراسیون', 'ناکجا', 'آباد', 'برد#بر']
url: https://www.farsnews.ir/news/14001117000518/اصولی-فدراسیون-فوتبال-جمهوری-اسلامی-ایران-هستیم-نه-جزیره-مستقل-با
--------------------------------------------------------
title: ['رئیس', ':', 'ایران', 'ثابت', 'کشور', 'مستقل', 'ایستاد#ایست']
url: https://www.farsnews.i

In [ ]:
def generate_champions_list():
  